# Lab 4

## 1. Setup

### Step 1: Copy content of lab-3 into lab-4

We will make use of the config, .env and packages installed from previous lab.

### Step 2: Create a `contracts` directory in `lab-4`

This directory will be used to store all contract files.

### Step 3: Create a `test` directory in `lab-4`

This directory will be used to store the test scripts

### Step 3: Install `chai` package

This package will be used for contract testing.

In [ ]:
cp -rp ~/github/suss/fin579/lab-3/. ~/github/suss/fin579/lab-4
cd ~/github/suss/fin579/lab-4
mkdir contracts
mkdir test
npm i -D chai

## Lab 4b: Create a contract

1. Create the contract `ERC20.sol` in the `contracts` directory.

    - Solidity files end with `.sol` extensions.
    - Solidity code begins its definition with a **pragma** directive. This contains instructions on the version of compiler required to compile a contract. In the following code snippet, the compiler is instructed to compile the code using version 0.8.8.
    - The following code defines a contract called ERC20 with an empty body.

        ```sol
        pragma solidity 0.8.8;

        contract ERC20 {

        }
        ```

## Lab 4c: Compile a contract

### Step 1. Compile the contract.

In [ ]:
hh compile

**NOTE**: The compiler will throw a warning saying that SPDX license identifier not provided in source file. You can ignore this warning.

### Step 2. Inspect the output

Hardhat, by default, compiles the output into the `artifacts` directory with the contract's name as sub directory name.

```sh
/artifacts
    /contracts
        /ERC20.sol
            ERC20.json
```

Open `/artifacts/contracts/ERC20.sol/ERC20.json`

* The `bytecode` property is the actual bytecode that will reside on the blockchain.
* The `deployedBytecode` property is the bytecode that contains the initialization logic, this property is seldom use unless used for estimating deployment cost.
* The `abi` property represents the `Application Binary Interface` which is used by external applications to interact with the smart contract on the blockchain. In this case, the `abi` is empty because the contracts does not contain any function at this point.

```js
{
    "_format": "hh-sol-artifact-1",
    "contractName": "ERC20",
    "sourceName": "contracts/ERC20.sol",
    "abi": [],
    "bytecode": "...",
    "deployedBytecode": "..."
}
```

## Lab 4d: Read-Only Functions

This session will implement the 6 readonly functions defined in the ERC20 specification (https://ethereum.org/en/developers/docs/standards/tokens/erc-20/):

-   function name() public **view** returns (string)
-   function symbol() public **view** returns (string)
-   function decimals() public **view** returns (uint8)

-   function totalSupply() public **view** returns (uint256)
-   function balanceOf(address \_owner) public **view** returns (uint256 balance)
-   function allowance(address \_owner, address \_spender) public **view** returns (uint256 remaining)

**NOTE:** Change the return type of **name()** and **symbol()** to return `string memory` instead of `string` otherwise the compiler with fail with `Data location must be "memory" or "calldata" for return parameter in function, but none was given` error.

### Step 1. Create the name() function.

This function returns the name of token as a string. We will create a state variable called \_name and return it via the function.

```js
contract ERC20 {
    string _name;
    function name() public view returns (string memory)
    {
        return _name;
    }
}
```

### Step 2. Create the symbol() function.

This function returns the symbol of the token (usually 3 to 4 characters) as a string. We will create a state variable \_symbol and return it via the function.

```js
contract ERC20 {
    ...
    string _symbol;

    ...
    function symbol() public view returns (string memory)
    {
        return _symbol;
    }
}
```

### Step 3. Create the decimal() function.

This function returns the number of decimal places this token's quantity can support. Since this value is conventionally 18, so we will return 18 and instead of creating a state variable to save on gas cost. We declare a function as **pure** keyword when it does not use any state variables.

```js
contract ERC20 {
    ...
    string _symbol;

    ...
    function decimals() public pure returns (uint8)
    {
        return 18;
    }
}
```

### Step 4. Create the totalSupply() function.

This function returns the total amount of ERC20 being minted as an unsigned 256-bit integer (**uint256**). We will create a state variable \_totalSupply and return it via the function.

```js
contract ERC20 {
    ...
    uint256 _totalSupply;

    ...
    function totalSupply() public view returns (uint256)
    {
        return _totalSupply;
    }
}
```

### Step 5. Create the balanceOf() function.

This function returns the balance of ERC20 belonging to a given **address** as an unsigned 256-bit integer (**uint256**). In order to track the balance of individual addresses, we will make use of a complex data type **mapping(address=>uint256)** for this. We will create a state variable \_balanceOf and return it via the function.

```js
contract ERC20 {
    ...
    mapping(address=>uint256) _balances;

    ...
    function balanceOf(address _owner) public view returns (uint256 balance)
    {
        return _balances[_owner];
    }
}
```

### Step 6. Create the allowance() function.

This function returns the withdrawal limit that has been authorised by the token owner to the withdrawer as an unsigned 256-bit integer (**uint256**).

The **owner** refers to the address where the tokens belongs to and the **spender** refers to the address that is authorised to withdraw the tokens from the owner address. In order to track the allowance for each owner and spender, we will make use of a double mapping **mapping(address=>mapping(address=>uint256))** for this.

```js
contract ERC20 {
    ...
    mapping(address=>mapping(address=>uint256)) _allowance;

    ...
    function allowance(address _owner, address _spender) public view returns (uint256 remaining)
    {
        return _allowance[_owner][_spender];
    }
}
```

### Step 7. Create constructor

At this stage, we have completed the creation of the 6 read-only functions. We will create a constructor that will initialize the state variables at the point of contract deployment before we start deploying and testing the contract.

* We will take in 4 parameters for this constructor: name, symbol, totalSupply, owner.
* This constructor will set the initial values of the state variables as well as to mint `totalSupply` amount of token and add it to the `owner` balance.

```js
contract ERC20 {

    ...

    constructor(string memory name_, string memory symbol_, uint256 totalSupply_, address owner)
    {
        _name=name_;
        _symbol=symbol_;
        _balances[owner]=totalSupply_;
    }
}

```

### Step 8. Compile the contract

```js
pragma solidity 0.8.8;

contract ERC20 {

    string _name;
    string _symbol;
    uint256 _totalSupply;
    mapping(address=>uint256) _balances;
    mapping(address=>mapping(address=>uint256)) _allowance;

    constructor(string memory name_, string memory symbol_, uint256 totalSupply_, address owner)
    {
        _name=name_;
        _symbol=symbol_;
        _balances[owner]=totalSupply_;
    }

    
    function name() public view returns (string memory)
    {
        return _name;
    }


    function symbol() public view returns (string memory)
    {
        return _symbol;
    }
    
    function decimals() public pure returns (uint8)
    {
        return 18;
    }

    function totalSupply() public view returns (uint256)
    {
        return _totalSupply;
    }

    function balanceOf(address _owner) public view returns (uint256 balance)
    {
        return _balances[_owner];
    }

    function allowance(address _owner, address _spender) public view returns (uint256 remaining)
    {
        return _allowance[_owner][_spender];
    }
}
    
```

In [ ]:
hh compile

## Lab 4e: Create unit test

### Step 1.  Create the test directory from the project directory

    ```sh
    md test; cd $_
    ```

    NOTE:
    The directory structure at this point should be

    ```
    /lab-6
        /contracts
        /test
        /node_modules
        /artifacts
    ```

### Step 2.  Create the file `testERC20.js` in `test` directory.
    
    We will use Hardhat test to run the test script. Hardhat Test uses the `mocha` framework and the `chai` assertion library.
    You can refer to here for the structure of the test scripts. (https://hardhat.org/tutorial/testing-contracts)
    The test is given an arbitrary scope name "Test ERC20".

```js
const { expect } = require("chai");
describe("Test ERC20", () => {});
```

### Step 3.  Initialise the test scope.

The test scope should initialise the `ERC20` token contract before each test. The name and symbol of the token is called `TUT`. And we will issue 1000 x 10^8 wei of `TUT` to your own address (`accounts[0]`). It should also initialise the `accounts` array from the development network wallet.

```js
describe("Test ERC20", () => {
    let erc20;
    let accounts;
    beforeEach(async () => {
        accounts = await ethers.getSigners();
        const factory = await ethers.getContractFactory("ERC20");
        erc20 = await factory.deploy(
            "TUT",
            "TUT",
            ethers.utils.parseUnits("1000", "ether"),
            accounts[0].address
        );
    });
});
```

### Step 4.  Create the test for name().

In this test, we will call the contract's name function, and check if the result from name is equal to `TUT` (which is the name given to it during deployment).

```js
const { expect } = require("chai");

describe("Test ERC20", () => {

    ...

    it("Should call name() and get TUT", async () => {
        let name = await erc20.name();
        expect(name).to.equals("TUT");
    });

});
```

### Step 5.  Run the test.

```js
const { expect } = require("chai");
describe("Test ERC20", () => {
    let erc20;
    let accounts;

    beforeEach(async () => {
        accounts = await ethers.getSigners();
        const factory = await ethers.getContractFactory("ERC20");
        erc20 = await factory.deploy(
            "TUT",
            "TUT",
            ethers.utils.parseUnits("1000", "ether"),
            accounts[0].address
        );
    });

    it("Should call name() and get TUT", async () => {
        let name = await erc20.name();
        expect(name).to.equals("TUT");
    });

});
```


In [ ]:
hh test

## Lab 4f: State-Changing Functions

This session will implement the 3 state-changing functions defined in the ERC20 specification (https://ethereum.org/en/developers/docs/standards/tokens/erc-20/):

-   function transfer(address \_to, uint256 \_value) public returns (bool success)
-   function transferFrom(address \_from, address \_to, uint256 \_value) public returns (bool success)
-   function approve(address \_spender, uint256 \_value) public returns (bool success)

### Step 1. Open `ERC20.sol`.

### Step 2. Create the transfer() function.

The purpose of the transfer function is to transfer certain amount of tokens indicated by (`_value`) to a recipient's address indicated by (`_to`). The transfer is performed by decreasing the your token account balance (`_balance`) and increasing the recipient's token account balance (`balance`).

a) Declare the function

```js
 function transfer(address _to, uint256 _value) external returns (bool success){
```

b) Use the balance from the transaction sender as `fromBalance`

```js
uint256 fromBalance = _balances[msg.sender];
```

c) Confirm that the transaction sender has sufficient balance to complete the transfer or revert with the message "Insufficient balance".

```js
require(fromBalance >= amount, "Insufficient balance");
```

d) Deduct the amount from sender's balance and increase the amount in recipient's balance.
        
```js
_balances[msg.sender] = fromBalance - amount;
_balances[to] += amount;
```
The final function should look like this:

```js
function transfer(address to, uint256 amount) external virtual returns (bool) {
    uint256 fromBalance = _balances[msg.sender];
    require(fromBalance >= amount, "insufficient balance");
    _balances[msg.sender] = fromBalance - amount;
    _balances[to] += amount;
    return true;
}
```


### Step 3. Create the transferFrom() function.

The transferFrom function allows a spender that has been previously authorised to withdraw certain amount of tokens from your wallet to transfer the tokens to a recipient. Unlike the `transfer` and `approve` function which is used by the token holder, the `transferFrom` function is called by the spender.

a) Declare the function

```js
function transferFrom(address _from, address _to, uint256 _value) external returns (bool success) {
```

b) The function should use the address of current transaction sender as the `spender`.

```js
    address spender = msg.sender;
```

c) It should find the authorised withdrawal amount (`currentAllowance`) to the `spender` by the owner.

```js
    uint256 currentAllowance = _allowance[_from][spender];
```

d) It should ensure that the authorized withdrawal amount (`currentAllowance`) is not less than the withdrawal amount (`value`). Otherwise, revert with the message (`Insufficient allowance`).

```js
    require(currentAllowance >= _value, "Insufficient allowance");
```

e) It should reduce the authorized withdrawal amount (`currentAllowance`) by the amount of tokens transferred (`value`)

```js
    _allowance[_from][spender] = currentAllowance - _value;
```

f) Proceed with the transfer, the transfer section is similar to the `transfer` function. Use the current balance of the owner as `fromBalance`.

```js
    uint256 fromBalance = _balances[_from];
```

g) Ensure that the transaction sender has sufficient balance to complete the transfer or revert with the message "Insufficient balance".

```js
    require(fromBalance >= _value, "Insufficient balance");
```

h) Deduct the amount from owner's balance and increase the amount in recipient's balance.

```js
    _balances[_from] = fromBalance - _value;
    _balances[_to] += _value;
```

i) The final function should look like this:

```js
function transferFrom(address _from, address _to, uint256 _value) external returns (bool success) {
    address spender = msg.sender;

    uint256 currentAllowance = _allowance[_from][spender];
    require(currentAllowance >= _value, "Insufficient allowance");
    _allowance[_from][spender] = currentAllowance - _value;

    uint256 fromBalance = _balances[_from];
    require(fromBalance >= _value, "Insufficient balance");
    _balances[_from] = fromBalance - _value;
    _balances[_to] += _value;

    return true;
}
```


### Step 4. Create the approve() function.

The purpose of the approve function is to grant permission to an address (`_spender`) to withdraw certain amount (`_value`) of tokens from your wallet by recording the authorized amount in the `_allowance` state variable. The approve function is called by the token holder, ie. only you have the permission to approve the spender to transfer from your wallet.

a) Declare the function

```js
function approve(address _spender, uint256 _value) external returns (bool success) {

}
```

b) Use the address of current transaction sender as the `owner`.

```js
   address owner = msg.sender;
```

c) Update the `_allowance` state variable with the authorized withdrawal amount `value`.

```js
   _allowance[owner][_spender] = _value;
```

**NOTE:**

- Imagine `_allowance` to be made up of 2 tables.
- The first table contains owners address with a reference to the second table.
- The second table contains spender addresses that is mapped to the authorized value.
- So effectively, `_allowance[owner][_spender]=_value` can be interpreted as

```
table1[owner] ===> table2[_spender] ===> _value
```

d) The final code should look like this.

```js
function approve(address _spender, uint256 _value) external returns (bool success) {
    address owner = msg.sender;
    _allowance[owner][_spender] = _value;
    return true;
}
```

### Step 5. Compile the code.

In [ ]:
hh compile

## Lab 4g: Create unit test

### Step 1. Open the test `testERC20.js` in the `test` directory.

### Step 2. Create the test for transfer().

In this test case, you are `accounts[0]`. You will send 1 wei of TUT to `accounts[1]` via the contract's `transfer` function.
The test is successful if the balance of `accounts[1]` before and after is increased by 1.

```js
it("Should transfer 1 TUT from accounts[0] to accounts[1]", async () => {
    const before = await erc20.balanceOf(accounts[1].address);

    // Transfer
    const response = await erc20.transfer(accounts[1].address, 1);
    const receipt = await response.wait();

    // Assert
    const after = await erc20.balanceOf(accounts[1].address);
    expect(before.add(1).toNumber()).equals(after.toNumber());
});
```

### Step 3. Create the test for approve() and transferFrom()

In the previous test, when calling the `transfer` function, you are using accounts[0]. But there is a little change to the flow when using approve and transferFrom. You as the token holder (`accounts[0]`) has to call the `approve` function. The spender (`accounts[1]`) has to be the one to call the `transferFrom` function.

#### Contract.connect() - Calling contract from a different account

When calling the contract from another account, it is not a simple matter of changing the address to another address. This involves unlocking the private key that belongs to a different account. The `connect()` function can be used to switch the contract to a different account.

Instead of this, which will call transferFrom using `accounts[0]`

```js
response = await erc20.transferFrom(
    accounts[0].address,
    accounts[1].address,
    1
);
```

Use this to connects to `accounts[1]` before making the call.

```js
response = await erc20
    .connect(accounts[1])
    .transferFrom(accounts[0].address, accounts[1].address, 1);
```

The test case for approve and transferFrom is shown below.

```js
it("Should approve and transferFrom 1 TUT from accounts[0] to accounts[1]", async () => {
    const before = await erc20.balanceOf(accounts[1].address);

    // Approve and TransferFrom
    let response = await erc20.approve(accounts[1].address, 1);
    let receipt = await response.wait();

    response = await erc20
        .connect(accounts[1])
        .transferFrom(accounts[0].address, accounts[1].address, 1);
    receipt = await response.wait();

    // Assert
    const after = await erc20.balanceOf(accounts[1].address);
    expect(before.add(1).toNumber()).equals(after.toNumber());
});
```

### Step 4. Run the tests


In [ ]:
hh test

## Optional: Running Tests in VS Code Test Explorer

1. VS Code must be opened in the lab directory otherwise the Test Explorer will return empty.

2. Create the file .mocharc

    ```json
    {
        "require": "hardhat/register",
        "timeout": 60000
    }
    ```

3. Open Test Explorer and refresh it. If its empty, you need to restart VS Code.

## Lab 4h: Events and Logs

This session will implement the 2 events defined in the ERC20 specification (https://ethereum.org/en/developers/docs/standards/tokens/erc-20/):

-   **event** Transfer(address indexed \_from, address indexed \_to, uint256 \_value);
-   **event** Approval(address indexed \_owner, address indexed \_spender, uint256 \_value);

### Step 1. Open 'ERC20.sol'

### Step 2. Declare the events.

```js
contract ERC20 {
    ...
    event Transfer(address indexed _from, address indexed _to, uint256 _value);
    event Approval(address indexed _owner, address indexed _spender, uint256 _value);
}
```

### Step 3. Add the call for **Transfer** event at the end of transfer().

```js
function transfer(address _to, uint256 _value) public returns (bool success) {
    ...
    emit Transfer(msg.sender, _to, _value);
    return true;
}
```

### Step 4. Add the call for **Transfer** event at the end of transferFrom() function on your own.

### Step 5. Add the call for **Approval** event at the end of approve() function on your own.

In [ ]:
hh compile

## Lab 4i: Create unit test

1. Open the test `testERC20.js` in the `test` directory.

2. Check for `Transfer` event

    ```js
    it("Should transfer 1 TUT and receive Transfer event", async () => {
        // Transfer
        const response = await erc20.transfer(accounts[1].address, 1);
        const receipt = await response.wait();

        const transfer = receipt.events.find((x) => x.event === "Transfer");
        expect(transfer.args._from).to.equals(accounts[0].address);
        expect(transfer.args._to).to.equals(accounts[1].address);
        expect(transfer.args._value.toNumber()).to.equals(1);
    });
    ```

3. Write the test to Check for `Transfer` and `Approval` event for the test `Should approve and transferFrom` on your own.

In [ ]:
hh test